# Own type hierarchy: a slightly bigger example

This is the key idea to keep in mind when building type hierarchies: things which subtype are inheriting behavior. You should setup your `abstract` types to mean the existance or non-existance of some behavior. For example: 

In [41]:
abstract type AbstractPerson end
abstract type AbstractStudent <: AbstractPerson end
abstract type AbstractTeacher <: AbstractPerson end

mutable struct Person <: AbstractPerson
  name::String    
end

mutable struct Student <: AbstractStudent
  name::String  
  grade::Int
  hobby::String
end

mutable struct MusicStudent <: AbstractStudent
  grade::Int
end

mutable struct Teacher <: AbstractTeacher
  name::String
  grade::Int
end

This can be interpreted as follows. At the top we have `AbstractPerson`. Our interface here is "a Person is someone who has a name which can be gotten by `get_name`". 

In [42]:
get_name(x::AbstractPerson) = x.name

get_name (generic function with 1 method)

Thus codes which are written for an `AbstractPerson` can "know" (by our informal declaration of the interface) that `get_name` will "just work" for its subtypes. However, notice that `MusicStudent` doesn't have a `name` field. This is because `MusicStudent`s just want to be named whatever the trendiest band is, so we can just replace the usage of the field by the action:

In [43]:
get_name(x::MusicStudent) = "Die Höhner"

get_name (generic function with 2 methods)

In this way, we can use `get_name` to get the name, and how it was implemented (whether it's pulling something that had to be stored from memory, or if it's something magically known in advance) does not matter. We can keep refining this: an `AbstractStudent` has a `get_hobby`, but a `MusicStudent`'s hobby is always `Music`, so there's not reason to store that data in the type and instead just have its actions implicitly "know" this. In non-trivial examples (like the range and `UniformScaling` above), this distinction by action and abstraction away from the actual implementation of the types allows for full optimization of generic codes.

### Small functions and constant propagation

The next question to ask is, does storing information in functions and actions affect performance? The answer is yes, in a good way! To see this, let's see what happens when we use these functions. To make it simpler, let's use a boolean function. Teachers are old and don't like music, while students do like music. But generally people like music. This means that:

In [36]:
likes_music(x::AbstractTeacher) = false
likes_music(x::AbstractStudent) = true
likes_music(x::AbstractPerson) = true

likes_music (generic function with 3 methods)

Now how many records would these people buy at a record store? If they don't like music, they will buy zero records. If they like music, then they will pick up a random number between 1 and 10. If they are a student, they will then double that (impulsive Millenials!).

In [35]:
function number_of_records(x::AbstractPerson)
    if !likes_music(x) 
      return 0
    end
    num_records = rand(10)
    if typeof(x) <: AbstractStudent
      return 2num_records
    else 
      return num_records
    end
end

number_of_records (generic function with 1 method)

Let's check the code that is created:

In [41]:
x = Teacher("Randy",11)
println(number_of_records(x))
@code_llvm number_of_records(x)

0

; Function number_of_records
; Location: In[35]:2
; Function Attrs: uwtable
define i64 @julia_number_of_records_35125(%jl_value_t addrspace(10)* nonnull align 8 dereferenceable(16)) #0 {
top:
  ret i64 0
}


The key thing to see from the typed code is that the "branches" (the `if` statements) all compiled away. Since types are known at compile time (remember, functions specialize on types), the dispatch of `likes_music` is known at compile-time. But this means, since the result is directly inferred from the dispatch, the boolean value `true/false` is known at compile time. This means that the compiler can directly infer the answer to all of these checks, and will use this information to skip them at runtime.

This is the distinction between compile-time information and runtime information. **At compile-time, what is known is:**

**1) The types of the inputs**

**2) Any types which can be inferred from the input types (via type-stability)**

**3) The function dispatches that will be internally called (from types which have been inferred)**

Note that what cannot be inferred by the compiler is the information in fields. Information in fields is strictly runtime information. This is easy to see since there is no way for the compiler to know that person's name was "Miguel": it is part of the type instance we just created.

Thus by putting our information into our functions and dispatches, we are actually giving the compiler more information to perform more optimizations. Therefore using this "action-based design", we are actually giving the compiler leeway to perform many extra optimizations on our code as long as we define our interfaces by the actions that are used. Of course, at the "very bottom" our algorithms have to use the fields of the types, but the full interface can then be built up using a simple set of functions which in many cases with replace runtime data with constants.

# Tim Holy's Trait Trick (advanced)

Since traits like "likes music" are compile-time information the compiler could in theory dispatch on them. Imagine you'd want to write a function
```julia
entertain(p::LikesMusic) = "turning the music on."
entertain(p::NotLikesMusic) = "better keep the music off. maybe play a game."
```
A way to do this is Tim Holy's Trait Trick. It's basically a **traits-based alternative to multiple inheritance**.

The trick was "invented" by Tim Holy in this [github issue](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633). See https://github.com/mauro3/Traits.jl#dispatch-on-traits for a detailed explanation and [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) for a convenience implementation.

Before we can understand THTT we have to understand what `Type{SomeType}` is and what `f(::Type{SomeType}) = "some function"` means. 

**Find out yourself by playing around with it (`isa` is your friend).**

[Solution](https://docs.julialang.org/en/v1/manual/types/#man-singleton-types-1)

In [57]:
# Music affinity
struct LikesMusic end
struct NotLikesMusic end

# trait function: map, say, person type to music affinity
likes_music(x::AbstractTeacher) = NotLikesMusic
likes_music(x::AbstractStudent) = LikesMusic
likes_music(x::AbstractPerson) = LikesMusic
likes_music(x::T) where T = error("Unknown music affinity for type $T")

likes_music (generic function with 4 methods)

No let's define the function `entertain` which dispatches on the "likes music" trait.

In [65]:
_entertain(::Type{LikesMusic}) = "turning the music on."
_entertain(::Type{NotLikesMusic}) = "better keep the music off. maybe play a game."

entertain(p) = _entertain(likes_music(p))

entertain (generic function with 2 methods)

In [59]:
c = Student("Susanne", 11, "soccer")
p = Person("Peter")
t = Teacher("Thomas", 10)
m = MusicStudent(9)

MusicStudent(9)

In [63]:
@code_typed entertain(c)

CodeInfo(
│4 1 ─     return "turning the music on."
) => String

In [64]:
@code_typed entertain(t)

CodeInfo(
│4 1 ─     return "better keep the music off. maybe play a game."
) => String

In [68]:
entertain(123.45)

ErrorException: Unknown music affinity for type Float64

Note that this trait system is general, it works for non boolean traits as well.

**The most important thing is however, that it is easily extendable.**

As we all know, dogs love music as well! Note that the only thing we have to do to make `entertain` work for them is to define their "likes music" trait function:

In [73]:
abstract type Animals end

mutable struct Dog <: Animals # single inheritance
    name::String
end

likes_music(x::Dog) = LikesMusic # has the "likes music" trait defined

ErrorException: invalid redefinition of constant Dog

In [74]:
wuff = Dog("Wolfgang")
entertain(wuff)

"turning the music on."

Ok, but this is students, teachers, and dogs. What about physics? Where could I possibly need this?

Think of 

* `entertain` as `perform_matrix_operation`
* `likes_music` as `select_method`.
* `LikesMusic` could be `FastMethod`.
* `NoLikesMusic` could be `SlowMethod`.

# Most important take home messages

* **Use types to move runtime information to compile-time.**
* **Use types and multiple dispatch to move `if` branches to compile time (and potentially compile them away). In my opinion, this also makes the code more readable.**